# Crypto Technical Screening

Here, we'll do some technical screening and explore ways to combine various technical indicators in our crypto universe

In [3]:
import requests
import pandas as pd
import numpy as np
import sys
sys.path.append('/Users/markwindsor/Desktop/atlas_trade')
from src.utils.technical_indicators import *

# Use your absolute path to our top 500 coins csv
top_coins_path = '/Users/markwindsor/Desktop/atlas_trade/src/data/top_coins.csv'

cryptoverse_df = pd.read_csv(top_coins_path)

# We'll load some price data into our dictionary 
cryptoverse_prices = {}

for index, row in cryptoverse_df.iterrows():
    id = row['id']
    name = row['cryptoName']
    symbol = row['symbol']

    # Retrieves OHLCV and mcap data 
    url = f"https://91j3dag4m9.execute-api.us-east-1.amazonaws.com/master/getCMCPriceHistory/{id}?startDate=2021-01-01"
    response = requests.get(url)
    data = response.json()
    table = []
    # Response gives us o,h,l,c,v,mcap keys which we need to change our column names like open, close etc...
    # because thats what our utility functions accept such as our technical indicator functions
    for entry in data["priceHistory"]:
        table.append([
            entry["o"],
            entry["h"],
            entry["l"],
            entry["c"],
            entry["v"],
            entry["mcap"]
        ])

    # Create a DataFrame from the extracted data
    columns = ["open", "high", "low", "close", "volume", "marketCap"]
    prices_df = pd.DataFrame(table, columns=columns)
    prices_df["log_returns"] = np.log(prices_df.close / prices_df.close.shift(1))
    prices_df["cumulative_log_returns"] = prices_df["log_returns"].cumsum()
    cryptoverse_prices[id] = prices_df


/Users/markwindsor/Desktop/atlas_trade/.venv/lib/python3.9/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/markwindsor/Desktop/atlas_trade/.venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:59: RuntimeWarning: invalid value encountered in accumulate
  return bound(*args, **kwds)
/Users/markwindsor/Desktop/atlas_trade/.venv/lib/python3.9/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/markwindsor/Desktop/atlas_trade/.venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:59: RuntimeWarning: invalid value encountered in accumulate
  return bound(*args, **kwds)
/Users/markwindsor/Desktop/atlas_trade/.venv/lib/python3.9/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Use

## RSI

Lets start with the RSI. Here, we'll explore screening the top cryptos with regards to the RSI. 0-30 = oversold, 30-70 = Neutral, 70-100 = Overbought

In [4]:
oversold = []
overbought = []

for index, row in cryptoverse_df.iterrows():
    id = row['id']
    name = row['cryptoName']
    symbol = row['symbol']

    rsi_df = relative_strength_index(cryptoverse_prices[id])
    last_row_rsi = rsi_df.iloc[-1]
    if (last_row_rsi['rsi'] < 30 and last_row_rsi['rsi'] > 0):
        oversold.append(f"id: {id}, symbol: {symbol}, name: {name}")
    if (last_row_rsi['rsi'] < 100 and last_row_rsi['rsi'] > 70):
        overbought.append(f"id: {id}, symbol: {symbol}, name: {name}")

print('RSI Below 30 (Oversold): ', oversold)
print('RSI 70 - 100 (Overbought): ', overbought)


RSI Below 30 (Oversold):  ['id: 4642, symbol: HBAR, name: HEDERA', 'id: 11840, symbol: OP, name: OPTIMISM', 'id: 6952, symbol: FRAX, name: FRAX', 'id: 7080, symbol: GALA, name: GALA', 'id: 4041, symbol: MX, name: MX TOKEN', 'id: 9258, symbol: XCH, name: CHIA', 'id: 11865, symbol: BONE, name: BONE SHIBASWAP', 'id: 12220, symbol: OSMO, name: OSMOSIS', 'id: 8677, symbol: XYM, name: SYMBOL', 'id: 2499, symbol: CHSB, name: SWISSBORG', 'id: 14783, symbol: MAGIC, name: MAGIC', 'id: 23254, symbol: CORE, name: CORE DAO', 'id: 5190, symbol: FLEX, name: FLEX', 'id: 5908, symbol: DKA, name: DKARGO', 'id: 11079, symbol: BRISE, name: BITGERT', 'id: 17591, symbol: NYM, name: NYM', 'id: 1762, symbol: ERG, name: ERGO', 'id: 9674, symbol: WILD, name: WILDER WORLD', 'id: 20886, symbol: ASTRAFER, name: ASTRAFER', 'id: 9868, symbol: XCAD, name: XCAD NETWORK', 'id: 21178, symbol: MRS, name: METARS GENESIS', 'id: 2840, symbol: QKC, name: QUARKCHAIN', 'id: 12147, symbol: SYN, name: SYNAPSE', 'id: 4189, symbol

## MACD

Lets explore the macd

In [5]:
macd_above_signal = []

for index, row in cryptoverse_df.iterrows():
    id = row['id']
    name = row['cryptoName']
    symbol = row['symbol']

    macd_df = macd(cryptoverse_prices[id])
    last_row_macd = macd_df.iloc[-1]
    if (last_row_macd['macd'] > last_row_macd['signal']):
        macd_above_signal.append(f"id: {id}, symbol: {symbol}, name: {name}")

print("MACD Line greater than signal line: ", macd_above_signal)

MACD Line greater than signal line:  ['id: 1, symbol: BTC, name: BITCOIN', 'id: 1027, symbol: ETH, name: ETHEREUM', 'id: 1839, symbol: BNB, name: BNB', 'id: 52, symbol: XRP, name: XRP', 'id: 3408, symbol: USDC, name: USD COIN', 'id: 8085, symbol: stETH, name: LIDO STAKED ETH', 'id: 2010, symbol: ADA, name: CARDANO', 'id: 74, symbol: DOGE, name: DOGECOIN', 'id: 5426, symbol: SOL, name: SOLANA', 'id: 18579, symbol: WTRX, name: WRAPPED TRON', 'id: 1958, symbol: TRX, name: TRON', 'id: 11419, symbol: TON, name: TONCOIN', 'id: 4943, symbol: DAI, name: DAI', 'id: 6636, symbol: DOT, name: POLKADOT', 'id: 3890, symbol: MATIC, name: POLYGON', 'id: 2, symbol: LTC, name: LITECOIN', 'id: 3717, symbol: WBTC, name: WRAPPED BITCOIN', 'id: 1831, symbol: BCH, name: BITCOIN CASH', 'id: 5805, symbol: AVAX, name: AVALANCHE', 'id: 1975, symbol: LINK, name: CHAINLINK', 'id: 512, symbol: XLM, name: STELLAR', 'id: 328, symbol: XMR, name: MONERO', 'id: 7083, symbol: UNI, name: UNISWAP', 'id: 3794, symbol: ATOM,

## Bollinger Bands



In [6]:
below_lower_band = []

for index, row in cryptoverse_df.iterrows():
    id = row['id']
    name = row['cryptoName']
    symbol = row['symbol']

    bb_df = bollinger_bands(cryptoverse_prices[id])
    last_row_bb = bb_df.iloc[-1]
    if (last_row_bb['close'] < last_row_bb['lower_band']):
        below_lower_band.append(f"id: {id}, symbol: {symbol}, name: {name}")

print("Previous close below lower band: ", below_lower_band)

Previous close below lower band:  ['id: 6952, symbol: FRAX, name: FRAX', 'id: 8677, symbol: XYM, name: SYMBOL', 'id: 21178, symbol: MRS, name: METARS GENESIS', 'id: 5919, symbol: MTRG, name: METER GOVERNANCE', 'id: 15447, symbol: LYRA, name: LYRA', 'id: 4950, symbol: LCX, name: LCX', 'id: 4064, symbol: USDK, name: USDK']


## Average True Range

The ATR is a moving average of the true ranges. The true range is the maximum value of:  
1. current high minus current low
2. absolute value of current high - previous close
3. absolute value of currenty low - previous close

Generally, people use 14 peroids




In [7]:
significant_drop = []

for index, row in cryptoverse_df.iterrows():
    id = row['id']
    name = row['cryptoName']
    symbol = row['symbol']

    atr_df = average_true_range(cryptoverse_prices[id])
    last_row_atr = atr_df.iloc[-1]

    atr_value = last_row_atr['ATR']
    last_close = last_row_atr['close']

    if last_close < atr_value * 2:
        significant_drop.append(f"id: {id}, symbol: {symbol}, name: {name}")

print("Significant drops based on ATR: ", significant_drop)

Significant drops based on ATR:  ['id: 3718, symbol: BTTOLD, name: BITTORRENT', 'id: 5190, symbol: FLEX, name: FLEX', 'id: 3089, symbol: AVINOC, name: AVINOC']


## Average Directional Index (ADX)

Measure the strength of a trend, irrespective of its direction. It doesn't tell you whether the trend is bullish or bearish, but rather how strong or weak the trend is.

0-25: Absent or weak trend
25-50: Strong trend
50-75: Very strong trend
75-100: Extremely strong trend


In [9]:
strong_trend = []

for index, row in cryptoverse_df.iterrows():
    id = row['id']
    name = row['cryptoName']
    symbol = row['symbol']

    adx_df = average_directional_index(cryptoverse_prices[id])
    last_row_adx = adx_df.iloc[-1]

    adx_value = last_row_adx['adx']
    last_close = last_row_adx['close']

    if adx_value > 75:
        strong_trend.append(f"id: {id}, symbol: {symbol}, name: {name}")

print("Cryptos in a strong trend: ", strong_trend)

Cryptos in a strong trend:  ['id: 2087, symbol: KCS, name: KUCOIN TOKEN', 'id: 5864, symbol: YFI, name: YEARN.FINANCE', 'id: 1567, symbol: XNO, name: NANO', 'id: 5455, symbol: TREX, name: TREXCOIN', 'id: 17591, symbol: NYM, name: NYM', 'id: 7959, symbol: vBUSD, name: VENUS BUSD', 'id: 3600, symbol: HUM, name: HUMANSCAPE', 'id: 18031, symbol: CCD, name: CONCORDIUM', 'id: 7150, symbol: FLM, name: FLAMINGO', 'id: 4944, symbol: TRB, name: TELLOR', 'id: 5072, symbol: RKN, name: RAKON', 'id: 5956, symbol: MCB, name: MUX PROTOCOL', 'id: 2492, symbol: ELA, name: ELASTOS', 'id: 22031, symbol: AGLA, name: ANGOLA', 'id: 6727, symbol: RSV, name: RESERVE', 'id: 8296, symbol: KSP, name: KLAYSWAP PROTOCOL', 'id: 3089, symbol: AVINOC, name: AVINOC']


## Stochastic Oscillator

Measure the position of the current closing price relative to the high and low range over a specified number of periods, typically 14.

In [12]:
oversold_so = []

for index, row in cryptoverse_df.iterrows():
    id = row['id']
    name = row['cryptoName']
    symbol = row['symbol']

    so_df = stochastic_oscillator(cryptoverse_prices[id])
    last_row_so = so_df.iloc[-1]

    percent_k = last_row_so['%K']
    percent_d = last_row_so['%D']

    if (percent_k) > 80:
        oversold_so.append(f"id: {id}, symbol: {symbol}, name: {name}")

print('Oversold according to Stochastic Oscillator: ', oversold_so)

Oversold according to Stochastic Oscillator:  ['id: 1958, symbol: TRX, name: TRON', 'id: 512, symbol: XLM, name: STELLAR', 'id: 11948, symbol: XRD, name: RADIX', 'id: 4157, symbol: RUNE, name: THORCHAIN', 'id: 1720, symbol: MIOTA, name: IOTA', 'id: 2502, symbol: HT, name: HUOBI TOKEN', 'id: 5830, symbol: NXM, name: NXM', 'id: 5804, symbol: DFI, name: DEFICHAIN', 'id: 8119, symbol: SFP, name: SAFEPAL', 'id: 8425, symbol: JASMY, name: JASMYCOIN', 'id: 1680, symbol: ANT, name: ARAGON', 'id: 23246, symbol: TOMI, name: TOMINET', 'id: 2570, symbol: TOMO, name: TOMOCHAIN', 'id: 5647, symbol: KDA, name: KADENA', 'id: 4948, symbol: CKB, name: NERVOS NETWORK', 'id: 7958, symbol: vUSDC, name: VENUS USDC', 'id: 6651, symbol: USDX, name: USDX [KAVA]', 'id: 5691, symbol: SKL, name: SKALE', 'id: 6958, symbol: ACH, name: ALCHEMY PAY', 'id: 1788, symbol: MTL, name: METAL DAO', 'id: 5455, symbol: TREX, name: TREXCOIN', 'id: 6950, symbol: PERP, name: PERPETUAL PROTOCOL', 'id: 2780, symbol: NKN, name: NKN